In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.grid_search import GridSearchCV

C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
import os

In [3]:
from sklearn.metrics import r2_score,mean_squared_error

In [4]:
os.chdir('../output_data')

In [5]:
df = pd.read_csv('train_v04.csv',header=0)

In [6]:
df.shape

(85799, 212)

In [7]:
os.chdir('../Models')

In [8]:
x = df.drop(['Close'],axis=1)

In [9]:
y = np.array(df['Close'])

In [10]:
final_model = GradientBoostingRegressor(max_depth=20,max_features=None,min_samples_leaf=.01,subsample= 1,random_state=42,n_estimators=250,learning_rate=.4)

In [11]:
final_model.fit(x,y)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.4, loss='ls', max_depth=20, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=0.01,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=250, presort='auto', random_state=42,
             subsample=1, verbose=0, warm_start=False)

In [13]:
pickle.dump(final_model,open('GBM_full.pickle','wb'))

In [14]:
y_pred = final_model.predict(x)

In [15]:
r2_score(y,y_pred)

0.9974859302148287

In [16]:
from sklearn.preprocessing import StandardScaler

In [17]:
scale = StandardScaler()
scale.fit(x)
scaled_x = scale.transform(x)

In [21]:
from sklearn.decomposition import PCA

In [22]:
pca = PCA(n_components=scaled_x.shape[1])
pca.fit(scaled_x)

PCA(copy=True, iterated_power='auto', n_components=211, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [23]:
explained_ratio = pca.explained_variance_ratio_.cumsum()

In [25]:
n_components =0
for i in range(scaled_x.shape[1]):
    n_components+=1
    if explained_ratio[i] >= .99:
        break

In [27]:
pca = PCA(n_components=n_components)
pca.fit(scaled_x)
pca_x = pca.transform(scaled_x)

In [28]:
os.chdir('../pickles/')

In [29]:
pickle.dump(scale,open('scale.pickle','wb'))
pickle.dump(pca,open('pca.pickle','wb'))

In [30]:
final_model = GradientBoostingRegressor(max_depth=20,max_features=None,min_samples_leaf=.01,subsample= 1,random_state=42,n_estimators=250,learning_rate=.4)

In [31]:
final_model.fit(pca_x,y)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.4, loss='ls', max_depth=20, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=0.01,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=250, presort='auto', random_state=42,
             subsample=1, verbose=0, warm_start=False)

In [32]:
os.chdir('../Models/')

In [33]:
pickle.dump(final_model,open('GBM_pca.pickle','wb'))

In [34]:
y_pred = final_model.predict(pca_x)

In [35]:
r2_score(y,y_pred)

0.9928858193109867